# Learning from Graphs

In [ ]:
import numpy as np
from PIL import Image
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

## The point cloud

In [ ]:
# Load point cloud data
pcd = o3d.io.read_point_cloud("../data/happy_vrip_res3.ply")

## Point clouds as meshes

In [ ]:
# Best parameters (lowest density variance) were found using grid search between [1, 20]
# for both parameters. Good normals are crucial for mesh reconstruction!
print("Computing surface normals...")
pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(8))
pcd.orient_normals_consistent_tangent_plane(8)

# Increase `depth` to get finer details
# This can take some time without a multi-core cpu
print("Making triangle mesh...")
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, n_threads=16)
mesh.remove_duplicated_vertices()
mesh.remove_unreferenced_vertices()
mesh.remove_duplicated_triangles()
mesh.remove_degenerate_triangles()
print(mesh)

# Reduce resolution to reduce file size
print("Decimating triangles...")
mesh = mesh.simplify_quadric_decimation(target_number_of_triangles=5000)
print(mesh)

vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)
x = vertices[:, 0]
y = vertices[:, 1]
z = vertices[:, 2]

fig = go.Figure(ff.create_trisurf(x=x, y=y, z=z,
                                  simplices=triangles,
                                  plot_edges=True,
                                  edges_color='black',
                                  colormap='Greys',
                                  show_colorbar=False).data)
fig.data[0].visible = False
fig.data[0].hoverinfo='none'
fig.data[1].hoverinfo='none'

fig.add_trace(go.Scatter3d(x=x,
                           y=y,
                           z=z,
                           mode="markers",
                           hovertemplate="<b>Vertex</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>",
                           hoverlabel=dict(bgcolor="white"),
                           marker=dict(size=4,
                                       color='red')))

# The figure factory creates two plots: 1. The triangle model, 2. The mesh.
# Visibility is set here changed in the visualization using the buttons.
buttons = [dict(label="mesh", method="update", args=[dict(visible=[False, True, True])]),
           dict(label="mesh w/ faces", method="update", args=[dict(visible=[True, True, True])]),
           dict(label="w/o mesh", method="update", args=[dict(visible=[True, False, False])])]

# Viewpoint
camera = dict(eye=dict(x=-0.1, y=0.01, z=0.06),
              up=dict(x=0, y=1, z=0),
              center=dict(x=0, y=0, z=0))

fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  showlegend=False,
                  height=1024,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera,
                  scene_dragmode="orbit",
                  updatemenus=[dict(buttons=buttons, x=0.2, y=1)])

In [ ]:
# Save figure
pio.write_html(fig,
               file=f"../_includes/figures/graph.html",
               full_html=False,
               include_plotlyjs='cdn')